In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from train import Trainer
from generator import *
from discriminator import GAN
from dataset import CocoStuffDataSet
import os, argparse, datetime, json

from utils import *
NUM_CLASSES = 11
SAVE_DIR = "../checkpoints" # Assuming this is launched from code/ subfolder.
batch_size = 1
print("This file is deprecated. RUN!")

In [ ]:
experiment_name = 'animal-batchnorm-50-nobnend'
experiment_dir = os.path.join(SAVE_DIR, experiment_name)

In [ ]:
HEIGHT = WIDTH = 128
val_dataset = CocoStuffDataSet(mode='val', supercategories=['animal'], height=HEIGHT, width=WIDTH, do_normalize=False)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)
train_loader = DataLoader(CocoStuffDataSet(mode='train', supercategories=['animal'], height=HEIGHT, width=WIDTH),
                            batch_size, shuffle=False)

In [ ]:
generator = SegNet16(NUM_CLASSES, use_bn=True)
discriminator = None
trainer = Trainer(generator, discriminator, train_loader, val_loader, \
                experiment_dir=experiment_dir, resume=True, load_iter=None)

In [ ]:
# for idx in (417, 77, 55):
# for idx in (16, 26, 27, 29, 37, 38, 39, 40, 42, 52, 77, 514):
for idx in (55,):
    savedir = os.path.join('./saved_images_and_masks', str(idx))
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    
    img, mask_gt, gt_visual = val_dataset[idx]
    torch.save(img, os.path.join(savedir, 'img.pk'))
    torch.save(mask_gt[-1], os.path.join(savedir, 'gt_mask.pk')) # only care about background class
    img = torch.unsqueeze(img, 0).to(trainer.device)
    gen_mask = convert_to_mask(trainer._gen(img))
    gen_mask = torch.squeeze(gen_mask, 0)
    torch.save(gen_mask[-1], os.path.join(savedir, 'gen_mask.pk'))  # only care about background class
    print("Finished saving to {}".format(savedir))
    plt.imshow(mask_gt[-1])
    plt.show()
    plt.imshow(gen_mask[-1])
    plt.show()
    